In [ ]:
from google.colab import drive
import pandas as pd
import json
import re
from scipy.stats import mannwhitneyu

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_output_df = pd.read_csv('/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/Test_output.csv')

In [ ]:
test_output_df = pd.read_csv('/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/Test_output.csv')

In [ ]:
with open("/content/drive/MyDrive/ML-Quiz-XRay-ReportGeneration/data/annotation_quiz_all.json", "r") as f:
    ground_truth = json.load(f)
test_ground_truth = ground_truth['test']

In [ ]:
def seperator(text):
  text = text.lower()
  patterns = {
      "lung": re.compile(r"lungs?\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "heart": re.compile(r"hearts?\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|lungs?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "mediastinal": re.compile(r"mediastinal\s*:\s*(.+?)(?=\n(?:bones?|lungs?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "bone": re.compile(r"bones?\s*:\s*(.+?)(?=\n(?:mediastinal|lungs?|hearts?|others)\s*:|$)", re.IGNORECASE | re.DOTALL),
      "others": re.compile(r"others\s*:\s*(.+?)(?=\n(?:mediastinal|bones?|lungs?|hearts?)\s*:|$)", re.IGNORECASE | re.DOTALL)
  }

  # Extracting the information using regex
  extracted_info = {key: (match.group(1).strip() if (match := pattern.search(text)) else "") for key, pattern in patterns.items()}
  return extracted_info

In [ ]:
def get_map(data):
  output = dict()
  for item in data:
    if 'report' in item:
      output[item['id']] = item['report']
    else:
      temp = dict()
      for key, value in item.items():
        if key in ['heart', 'bone', 'mediastinal', 'other', 'lung']:
          temp[key] = value
      output[item['id']] = temp
  return output

In [ ]:
def get_region(df, region):
  predicted = []
  ground_truth = []
  for index, row in df.iterrows():
    if region in row['g_truth'].keys():
      predicted.append(row['formatted_output'][region])
      ground_truth.append(row['g_truth'][region])
  return predicted, ground_truth

In [ ]:
test_output_df['formatted_output'] = test_output_df['output'].apply(seperator)

In [ ]:
test_output_map = get_map(ground_truth['test'])

In [ ]:
test_output_df['g_truth'] = test_output_df['id'].apply(lambda x: test_output_map[x])

In [ ]:
predicted, g_truth = get_region(test_output_df, 'bone')

In [ ]:
(len([item for item in predicted if ('are not clearly visible' in item or 'to be intact without any signs of fracture or dislocation' in item)])/len(predicted))*100

55.932203389830505

In [ ]:
observation_zero = []
observation_nonzero = []
for p, g in zip(predicted, g_truth):
  if len(g) <=1:
    target_array = observation_zero
  else:
    target_array = observation_nonzero
  if 'are not clearly visible' in p or 'to be intact without any signs of fracture or dislocation' in p:
    target_array.append(0)
  else:
    target_array.append(1)


In [ ]:
mannwhitneyu(observation_zero, observation_nonzero, alternative='less')

MannwhitneyuResult(statistic=380.5, pvalue=0.2754716822647637)

In [ ]:
(sum([1 for item in ground_truth['train'] if len(item['report']['mediastinal']) <= 1])/len(ground_truth['train'])) *100

21.846302561623972